main.csv - основная информация о предприятиях общественного питания (скачать)
- ID - уникальный номер объекта
- Name - наименование
- Address - адрес
---
net_property.csv - является ли предприятие сетевым (скачать)
- ID - уникальный номер объекта
- IsNetObject - метка, которая указывает на то, является ли предприятие сетевым
---
seats_property.csv - количество посадочных мест (скачать)
- ID - уникальный номер объекта
- SeatsCount - количество посадочных мест
- ver_ID - номер версии записи (чем выше, тем запись более актуальная)
---
type_property.csv - информация о специализации: столовая, закусочная, кафе, иное (скачать)
- ID - уникальный номер объекта
- TypeObject - специализация (тип) объекта
- is_actual - является ли запись о специализации актуальной. 0 - не актуальная, 1 - актуальная

In [66]:
import pandas as pd
import numpy as np

In [2]:
df_main = pd.read_csv('6_main.csv')

In [3]:
df_net_property = pd.read_csv('6_net_property.csv')

In [4]:
df_seats_property = pd.read_csv('6_seats_property.csv')

In [5]:
df_type_property = pd.read_csv('6_type_property.csv')

## Урок 6.3

### Задание 1
Определите процент объектов у которых запись о специализации отсутствует или не является актуальной. Получившийся процент округлите до целых.

In [7]:
df_1 = pd.merge(df_main, df_type_property, how='left')
df_1.head()

,ID,Name,Address,TypeObject,is_actual
0,20988,ГУЗ,"город Москва, улица Казакова, дом 15",столовая,True
1,24828,Кафе Халяль,"город Москва, 8-я улица Текстильщиков, дом 5, ...",NaN,NaN
2,358126,Scarface,"Российская Федерация, город Москва, внутригоро...",кафе,True
3,373052,Кулинариум,"Российская Федерация, город Москва, внутригоро...",кафетерий,False
4,303239,One price coffee,"Российская Федерация, город Москва, внутригоро...",NaN,NaN


In [11]:
round((df_1.shape[0] - df_1.is_actual.sum()) / df_1.shape[0] * 100)

67

### Задание 2
Подтяните к исходному списку (main.csv) последние данные о числе посадочных мест для каждого объекта. Сколько всего посадочных мест получилось?

In [27]:
df_seats_property.groupby('ID').max('ver_ID')

,SeatsCount,ver_ID
ID,,
5882,75,2
19627,20,2
19657,85,2
19678,46,1
19681,24,1
...,...,...
375905,34,1
375933,5,2
375940,10,2


In [34]:
df_2 = pd.merge(df_main, df_seats_property.groupby('ID').max('ver_ID'), left_on = 'ID', right_index=True)
df_2.head()

,ID,Name,Address,SeatsCount,ver_ID
3,373052,Кулинариум,"Российская Федерация, город Москва, внутригоро...",4,1
4,303239,One price coffee,"Российская Федерация, город Москва, внутригоро...",5,2
6,342854,МЯСОЕД STEAK&BRISKET,"город Москва, проспект Вернадского, дом 86В",5,2
7,363640,Шаурма-выпечка,"Российская Федерация, город Москва, внутригоро...",5,2
11,254675,Timeless,"Российская Федерация, город Москва, внутригоро...",31,2


In [35]:
df_2.SeatsCount.sum()

188241

In [37]:
pd.merge(
        df_main, 
        df_seats_property.groupby('ID').max('ver_ID'), 
        left_on = 'ID', 
        right_index=True
)\
.SeatsCount\
.sum()

188241

### Задание 3
Сформируйте итоговую выгрузку в формате csv (разделитель запятая), которая будет содержать все объекты общепита из файла main.csv. Подтяните признак сетевой/не сетевой и актуальную специализацию. Отсортируйте по специализации  в алфавитном порядке. Внутри каждой специализации сортировка должна идти по убыванию ID. Объекты у которых отсутствует специализация должны идти в начале выгрузки (и тоже по убыванию ID).  
Колонки, которые должны быть в csv файле: ID,Name,Address,IsNetObject,TypeObject,is_actual.

In [4]:
df_main = pd.read_csv('6_main.csv')

In [5]:
df_net_property = pd.read_csv('6_net_property.csv')

In [7]:
df_type_property = pd.read_csv('6_type_property.csv')

In [23]:
df_answer = \
    pd.merge(
            pd.merge(df_main, df_net_property, how='left'), 
            df_type_property[df_type_property.is_actual], 
            how='left').\
    sort_values(['TypeObject', 'ID'], ascending=[True, False], na_position='first')
df_answer.head()

,ID,Name,Address,IsNetObject,TypeObject,is_actual
4912,375956,Крошка Картошка,"город Москва, Люблинская улица, дом 169, корпус 2",да,NaN,NaN
5063,375954,Кофе,"Российская Федерация, город Москва, внутригоро...",NaN,NaN,NaN
7804,375834,Донеретто,"город Москва, улица Лестева, дом 6",нет,NaN,NaN
493,375830,Шаурма,"город Москва, улица Вертолётчиков, дом 2А, кор...",NaN,NaN,NaN
5521,375822,Кафе,"Российская Федерация, город Москва, внутригоро...",NaN,NaN,NaN


In [24]:
df_answer.to_csv('6_3_3.csv', index=False)

## Урок 6.4

### Задание 6.4.1
Разбейте все сетевые объекты общепита со специализацией кафе на 4 группы (по количеству посадочных мест):

1. 0 - 5
2. 6 - 15 
3. 16 - 25
4. 26+  

В какой группе больше всего объектов?

In [3]:
df_main = pd.read_csv('6_main.csv')

In [4]:
df_type_property = pd.read_csv('6_type_property.csv')

In [8]:
df_seats_property = pd.read_csv('6_seats_property.csv')

In [50]:
df_net_property = pd.read_csv('6_net_property.csv')

In [71]:
df_answer = \
    pd.merge(
        pd.merge(
            pd.merge(
                df_main, 
                df_type_property[df_type_property.is_actual], 
                how='left'), 
            df_seats_property, 
            how='left'), 
        df_net_property, 
        how='left').\
    query("TypeObject=='кафе' & IsNetObject=='да' & SeatsCount > 0").\
    groupby('ID').\
    max('ver_ID')

In [72]:
df_answer['SeatsCountGroup'] = pd.cut(df_answer.SeatsCount, [0, 5, 15, 25, df_answer.SeatsCount.max()], labels=['0 - 5', '6 - 15', '16 - 25', '26+'])

In [79]:
df_answer.sample(10)

,SeatsCount,ver_ID,SeatsCountGroup
ID,,,
260577,55.0,2.0,26+
192911,95.0,2.0,26+
342389,54.0,1.0,26+
20182,35.0,1.0,26+
138495,23.0,2.0,16 - 25
279733,15.0,2.0,6 - 15
149196,20.0,2.0,16 - 25
288642,65.0,2.0,26+
360088,15.0,1.0,6 - 15


In [86]:
df_answer.groupby('SeatsCountGroup').SeatsCount.count()

SeatsCountGroup
0 - 5       1
6 - 15     12
16 - 25    10
26+        46
Name: SeatsCount, dtype: int64

In [6]:
pd.read_csv('6_main.csv').head(1)

,ID,Name,Address
0,20988,ГУЗ,"город Москва, улица Казакова, дом 15"


In [7]:
pd.read_csv('6_net_property.csv').head(1)

,ID,IsNetObject
0,369614,нет


In [8]:
pd.read_csv('6_seats_property.csv').head(1)

,ID,SeatsCount,ver_ID
0,345181,0,1


In [9]:
pd.read_csv('6_type_property.csv').head(1)

,ID,TypeObject,is_actual
0,302619,закусочная,True
